In [1]:
import requests
import json
import pandas as pd


In [2]:
url ="http://publicplansdata.org/api/"
#http://publicplansdata.org/api/?q=QVariables&variables=fy,PlanName,PercentReqContPaid,InvestmentReturn_5yr,ActFundedRatio_GASB&filterfystart=2000&filterfyend=2019&format=json

In [3]:
params = {
    "q": "ListDataSets",
     "format": "json"
    
}


In [4]:
response = requests.get(url, params = params)
print(response.url)

http://publicplansdata.org/api/?q=ListDataSets&format=json


In [5]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-27 23:49:27',
  'q': 'listdatasets',
  'params': 'q=ListDataSets&format=json',
  'recordcount': 34},
 {'id': '3',
  'table_name': 'pensiongasbschedules',
  'table_display_name': 'Funding'},
 {'id': '4',
  'table_name': 'pensiongasbassumptions',
  'table_display_name': 'Assumptions and Methods'},
 {'id': '5',
  'table_name': 'pensionincomestatement',
  'table_display_name': 'Income Statement'},
 {'id': '7',
  'table_name': 'pensioninvestmentreturn',
  'table_display_name': 'Investment Returns'},
 {'id': '8',
  'table_name': 'pensionactuarialcosts',
  'table_display_name': 'Actuarial Costs'},
 {'id': '9',
  'table_name': 'pensionprovisions',
  'table_display_name': 'Plan Provisions'},
 {'id': '10',
  'table_name': 'pensionmembership',
  'table_display_name': 'Membership'},
 {'id': '11',
  'table_name': 'pensionreportingdates',
  'table_display_name': 'Plan Reporting'},
 {'id': '12',
  'table_name': 'pensioneegroupbasics',
  'table_display_name': 'Empl

In [6]:
#Make sure to use QDataset, NOT QDataSet.  Makes a big difference
paramsratios = {
    "q": "QDataset",
    "dataset": "pensiongasbschedules",
     "format": "json",
    #"variables": 
     #"v": variables,
}
paramsbasicplandata = {
    "q": "QDataset",
    "dataset": "pensionplanbasics",
     "format": "json",
    #"variables": 
     #"v": ,
}


In [7]:
ratiosresponse = requests.get(url, params = paramsratios)
#print(ratiosresponse.url)
plandataresponse = requests.get(url,params=paramsbasicplandata)

In [8]:
fundedratiosdata = ratiosresponse.json()
plandata = plandataresponse.json()

In [9]:
funded_ratios_full = pd.DataFrame(fundedratiosdata[1:])
plan_data_full = pd.DataFrame(plandata[1:])

In [10]:
funded_ratios_full.columns

Index(['ActAssets_est', 'ActAssets_GASB', 'ActFundedRatio_est',
       'ActFundedRatio_GASB', 'ActFundedRatio_GASB67', 'ActLiabilities_est',
       'ActLiabilities_GASB', 'ActLiabilities_other', 'ADEC', 'EEGroupID',
       'fy', 'NetPensionLiability', 'NetPosition', 'notes_GASBSchedules',
       'payroll', 'PercentReqContPaid', 'ppd_id', 'RequiredContribution',
       'RequiredContribution_est', 'source_GASBSchedules', 'TierID',
       'TotalPensionLiability', 'UAAL_GASB'],
      dtype='object')

In [11]:
#remove rows with empty funded ratio data
funded_ratio_df=funded_ratios_full[['ppd_id','fy','ActFundedRatio_GASB','ActFundedRatio_est']].dropna()
funded_ratio_df.shape

(4032, 4)

In [12]:
plan_data_full.columns

Index(['AdministeringGovt', 'BenefitsWebsite', 'CostSharing', 'CostStructure',
       'DataEndYear', 'DataStartYear', 'EmployeeTypeCovered', 'EmployerType',
       'FiscalYearType', 'GovtName', 'InPFS', 'notes_PlanBasics', 'PlanClosed',
       'PlanFullName', 'PlanInceptionYear', 'PlanName', 'PlanType',
       'PlanYearClosed', 'ppd_directory_listed', 'ppd_id', 'SocSecCovered',
       'SocSecCovered_verbatim', 'source_PlanBasics', 'StateAbbrev',
       'StateName', 'system_id'],
      dtype='object')

In [13]:
plan_data_df=plan_data_full[['ppd_id','PlanName','StateAbbrev','PlanClosed','PlanYearClosed',]]
plan_data_df.head()

,ppd_id,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,Alabama ERS,AL,0,None
1,2,Alabama Teachers,AL,0,None
2,3,Alaska PERS,AK,1,2006
3,4,Alaska Teachers,AK,1,2006
4,5,Arizona Public Safety,AZ,0,None


In [14]:
plan_test_data_df=pd.merge(funded_ratio_df,plan_data_df,how="inner",on="ppd_id")
plan_test_data_df['ActFundedRatio_GASB'] = plan_test_data_df['ActFundedRatio_GASB'].astype(float)
plan_test_data_df.shape

(4032, 8)

In [15]:
plan_test_data_df.head(50)

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,2001,1.002,0.000,Alabama ERS,AL,0,None
1,1,2002,0.954,0.000,Alabama ERS,AL,0,None
2,1,2003,0.911,0.000,Alabama ERS,AL,0,None
3,1,2004,0.897,0.000,Alabama ERS,AL,0,None
4,1,2005,0.840,0.000,Alabama ERS,AL,0,None
5,1,2006,0.811,0.000,Alabama ERS,AL,0,None
6,1,2007,0.790,0.000,Alabama ERS,AL,0,None
7,1,2008,0.757,0.000,Alabama ERS,AL,0,None
8,1,2009,0.722,0.000,Alabama ERS,AL,0,None
9,1,2010,0.682,0.000,Alabama ERS,AL,0,None


In [16]:
plan_data_2020=plan_test_data_df[(plan_test_data_df['fy']=='2020')]
plan_data_2020

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
19,1,2020,0.68200,0.000,Alabama ERS,AL,0,None
39,2,2020,0.69400,0.000,Alabama Teachers,AL,0,None
59,3,2020,0.63700,0.000,Alaska PERS,AK,1,2006
79,4,2020,0.75300,0.000,Alaska Teachers,AK,1,2006
99,5,2020,0.47006,0.000,Arizona Public Safety,AZ,0,None
...,...,...,...,...,...,...,...,...
3917,198,2020,0.88900,0.000,Oklahoma Municipal Employees,OK,0,None
3937,199,2020,0.75800,0.000,Vermont Municipal Employees,VT,0,None
3957,200,2020,0.77620,0.000,St. Louis Employees,MO,0,None
3992,202,2020,0.51200,0.000,Jersey City Municipal Employees,NJ,0,None


In [17]:
plan_data_2020.sort_values(by='ActFundedRatio_GASB', ascending=False)[0:10]

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
2416,117,2020,1.11000,0.000,Washington LEOFF Plan 2,WA,0,None
2728,134,2020,1.00800,0.000,Oklahoma Police,OK,0,None
2122,101,2020,1.00100,0.000,South Dakota RS,SD,0,None
1576,78,2020,0.99600,0.000,New York State Teachers,NY,0,None
3331,166,2020,0.99200,0.000,Montgomery County Maryland ERS,MD,0,None
3548,178,2020,0.96100,0.000,Oklahoma City ERS,OK,0,None
2436,119,2020,0.96000,0.000,Washington PERS 2/3,WA,0,None
1674,83,2020,0.95242,0.000,NY State & Local ERS,NY,0,None
3175,158,2020,0.95100,0.000,Nashville-Davidson ERS,TN,0,None
2863,141,2020,0.93920,0.000,Los Angeles Water and Power,CA,0,None


In [18]:
plan_data_20yr=plan_test_data_df[(plan_test_data_df['ActFundedRatio_GASB']>=.9)]
plan_data_20yr

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,2001,1.0020,0.000,Alabama ERS,AL,0,None
1,1,2002,0.9540,0.000,Alabama ERS,AL,0,None
2,1,2003,0.9110,0.000,Alabama ERS,AL,0,None
20,2,2001,1.0140,0.000,Alabama Teachers,AL,0,None
21,2,2002,0.9740,0.000,Alabama Teachers,AL,0,None
...,...,...,...,...,...,...,...,...
3963,201,2006,1.0857,0.000,Dallas ERS,TX,0,None
3964,201,2007,1.0920,0.000,Dallas ERS,TX,0,None
3965,201,2008,0.9617,0.000,Dallas ERS,TX,0,None
3966,201,2009,0.9497,0.000,Dallas ERS,TX,0,None
